In [ ]:
#prepare for RDS connection
# !pip install psycopg2

In [1]:
# dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine, text
import psycopg2
from getpass import getpass
from sklearn.preprocessing import LabelEncoder

/Users/joeymyers/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Set up connection between notebook and database

In [2]:
# create SQL Alchemy connection from RDS
secret = getpass('Enter the secret value: ')

args ={
    'host':"ogdataset.c11hekhsylui.us-west-1.rds.amazonaws.com",
    'port':'5432',
    'database':"og_dataset",
    'user':"attritionProject",
    'password':secret
}

engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{database}".format(**args))
connection = engine.connect()

Enter the secret value: ········


In [20]:
# read in table from sql 
attrition_df = pd.read_sql('SELECT * FROM og_dataset', connection)

In [4]:
# view dataframe
attrition_df.head()

,Age,Attrition,Business Travel,Daily Rate,Department,Distance from Home,Education Level,Education Field,Employee Count,Employee Number,...,Relationship Satisfaction,Standard Hours,Stock Option Level,Total Working Years,Training Times Last Year,Work Life Balance,Years at Company,Years In Current Role,Years Since Last Promotion,Years With Current Manager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


# Preprocess Data

In [5]:
# determine if there are any missing values and data types for columns
attrition_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Age                         1470 non-null   int64 
 1   Attrition                   1470 non-null   object
 2   Business Travel             1470 non-null   object
 3   Daily Rate                  1470 non-null   int64 
 4   Department                  1470 non-null   object
 5   Distance from Home          1470 non-null   int64 
 6   Education Level             1470 non-null   int64 
 7   Education Field             1470 non-null   object
 8   Employee Count              1470 non-null   int64 
 9   Employee Number             1470 non-null   int64 
 10  Environment Satisfaction    1470 non-null   int64 
 11  Gender                      1470 non-null   object
 12  Hourly Rate                 1470 non-null   int64 
 13  Job Involvement             1470 non-null   int6

In [6]:
# column names
attrition_df.columns

Index(['Age', 'Attrition', 'Business Travel', 'Daily Rate', 'Department',
       'Distance from Home', 'Education Level', 'Education Field',
       'Employee Count', 'Employee Number', 'Environment Satisfaction',
       'Gender', 'Hourly Rate', 'Job Involvement', 'Job Level', 'Job Role',
       'Job Satisfaction', 'Marital Status', 'Monthly Income', 'Monthly Rate',
       'Number Companies Worked', 'Over 18', 'Overtime', 'Percent Salary Hike',
       'Performance Rating', 'Relationship Satisfaction', 'Standard Hours',
       'Stock Option Level', 'Total Working Years', 'Training Times Last Year',
       'Work Life Balance', 'Years at Company', 'Years In Current Role',
       'Years Since Last Promotion', 'Years With Current Manager'],
      dtype='object')

In [7]:
# ambiguous/unnecessary columns - only going to keep MonthlyIncome
attrition_df[["Daily Rate", "Hourly Rate", "Monthly Income", "Monthly Rate", "Standard Hours", "Over 18"]]

,Daily Rate,Hourly Rate,Monthly Income,Monthly Rate,Standard Hours,Over 18
0,1102,94,5993,19479,80,Y
1,279,61,5130,24907,80,Y
2,1373,92,2090,2396,80,Y
3,1392,56,2909,23159,80,Y
4,591,40,3468,16632,80,Y
...,...,...,...,...,...,...
1465,884,41,2571,12290,80,Y
1466,613,42,9991,21457,80,Y
1467,155,87,6142,5174,80,Y
1468,1023,63,5390,13243,80,Y


Drop unnecessary columns

In [8]:
# these columns are either unique identifiers, ambiguous or constant thru each column
columns_to_drop = ["Daily Rate", "Hourly Rate", "Monthly Rate", "Standard Hours", "Employee Count", 
                   "Employee Number", "Over 18"]

In [9]:
attrition_df.drop(columns=columns_to_drop, inplace=True)
attrition_df.head()

,Age,Attrition,Business Travel,Department,Distance from Home,Education Level,Education Field,Environment Satisfaction,Gender,Job Involvement,...,Performance Rating,Relationship Satisfaction,Stock Option Level,Total Working Years,Training Times Last Year,Work Life Balance,Years at Company,Years In Current Role,Years Since Last Promotion,Years With Current Manager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,Female,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,Male,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,Male,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,Female,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,Male,3,...,3,4,1,6,3,3,2,2,2,2


In [10]:
# number of columns now down from 35 to 28
len(attrition_df.columns)

28

Send cleaned Dataframe to the database

In [11]:
# Write cleaned attrition to table in RDS, drop index
attrition_df.to_sql('cleaned_data', engine, index=False)

Send cleaned Dataframe to csv for Tableau

In [12]:
# send cleaned data to csv for Tableau use
attrition_df.to_csv("clean_data.csv", index=False)

# Encoding

Making two separate lists of columns that need to be encoded and join them

In [13]:
# making a list of columns with object data types
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()
attrition_cat

['Attrition',
 'Business Travel',
 'Department',
 'Education Field',
 'Gender',
 'Job Role',
 'Marital Status',
 'Overtime']

In [14]:
# number of unique items in each column
attrition_df[attrition_cat].nunique()

Attrition          2
Business Travel    3
Department         3
Education Field    6
Gender             2
Job Role           9
Marital Status     3
Overtime           2
dtype: int64

In [15]:
# amount of columns that will be encoded: 8
len(attrition_cat)

8

In [16]:
# create OrdinalEncoder insatnce
enc = LabelEncoder()

# loop through each column with a categorical data type and encode it
for column in attrition_cat:
    attrition_df[column] = enc.fit_transform(attrition_df[column])
    
attrition_df.head()

,Age,Attrition,Business Travel,Department,Distance from Home,Education Level,Education Field,Environment Satisfaction,Gender,Job Involvement,...,Performance Rating,Relationship Satisfaction,Stock Option Level,Total Working Years,Training Times Last Year,Work Life Balance,Years at Company,Years In Current Role,Years Since Last Promotion,Years With Current Manager
0,41,1,2,2,1,2,1,2,0,3,...,3,1,0,8,0,1,6,4,0,5
1,49,0,1,1,8,1,1,3,1,2,...,4,4,1,10,3,3,10,7,1,7
2,37,1,2,1,2,2,4,4,1,2,...,3,2,0,7,3,3,0,0,0,0
3,33,0,1,1,3,4,1,4,0,3,...,3,3,0,8,3,3,8,7,3,0
4,27,0,2,1,2,1,3,1,1,3,...,3,4,1,6,3,3,2,2,2,2


In [17]:
# determine if encoding was sucessful    
attrition_df.head()

,Age,Attrition,Business Travel,Department,Distance from Home,Education Level,Education Field,Environment Satisfaction,Gender,Job Involvement,...,Performance Rating,Relationship Satisfaction,Stock Option Level,Total Working Years,Training Times Last Year,Work Life Balance,Years at Company,Years In Current Role,Years Since Last Promotion,Years With Current Manager
0,41,1,2,2,1,2,1,2,0,3,...,3,1,0,8,0,1,6,4,0,5
1,49,0,1,1,8,1,1,3,1,2,...,4,4,1,10,3,3,10,7,1,7
2,37,1,2,1,2,2,4,4,1,2,...,3,2,0,7,3,3,0,0,0,0
3,33,0,1,1,3,4,1,4,0,3,...,3,3,0,8,3,3,8,7,3,0
4,27,0,2,1,2,1,3,1,1,3,...,3,4,1,6,3,3,2,2,2,2


In [19]:
# Write encoded attrition dataframe to table in RDS, drop index
attrition_df.to_sql('new_encoded_data', engine, index=False, method='multi')